In [2]:
from qiskit import *
from qiskit_aer import AerSimulator
import numpy as np
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_experiments import *
from qiskit.quantum_info import state_fidelity

#### Estabelecimento do backend

In [3]:
backend = AerSimulator()

#### Funcao de tomografia e teleporte integrados

In [ ]:
def function_tel_tomography(n=0, shots=1000):
    # Registradores quânticos
    Alice = QuantumRegister(1, 'Alice')
    Bob = QuantumRegister(1, 'Bob')
    Secret = QuantumRegister(1, 'Secret')
    c = ClassicalRegister(3, 'c')  # c[0] = Secret, c[1] = Alice, c[2] = tomografia

    if n > 0:
        ext = QuantumRegister(n, 'ext')
        circuit = QuantumCircuit(Secret, Alice, Bob, ext, c)
    else:
        circuit = QuantumCircuit(Secret, Alice, Bob, c)

    # EPR
    circuit.h(Alice)
    circuit.cx(Alice, Bob)
    circuit.barrier()

    # Swaps
    if n > 0:
        circuit.swap(Bob[0], ext[0])
        for i in range(1, n):
            circuit.swap(ext[i - 1], ext[i])
        final_target = ext[n - 1]
    else:
        final_target = Bob[0]

    # Estado secreto: H|0> = |+>
    circuit.barrier()
    circuit.h(Secret)
    circuit.barrier()

    # Entrelançamento e medições
    circuit.cx(Secret, Alice)
    circuit.h(Secret)
    circuit.barrier()

    circuit.measure(Alice, c[0])
    circuit.measure(Secret, c[1])
    circuit.barrier()

    # Correções condicionais baseadas nas medições
    with circuit.if_test((c[0], 1)):  # Se Secret medir 1 → aplicar X
        circuit.x(final_target)

    with circuit.if_test((c[1], 1)):  # Se Alice medir 1 → aplicar Z
        circuit.z(final_target)

    #Hadamard no final ciruito
    circuit.h(final_target)

    # =============== TOMOGRAFIA DIRETA ===============
    pauli_list = [
        np.eye(2),
        np.array([[0.0, 1.0], [1.0, 0.0]]),       # X
        np.array([[0, -1.0j], [1.0j, 0.0]]),      # Y
        np.array([[1.0, 0.0], [0.0, -1.0]])       # Z
    ]
    expectation_values = {}
    all_counts = {}

    for base in ['X', 'Y', 'Z']:
        circ = circuit.copy()

        # Aplica a rotação de base antes da medição
        if base == 'X':
            circ.h(final_target)
        elif base == 'Y':
            circ.sdg(final_target)
            circ.h(final_target)

        # Mede o qubit final no registrador c[2]
        circ.measure(final_target, c[2])

        transpiled = transpile(circ, backend)
        result = backend.run(transpiled, shots=shots, backend=backend).result()
        counts = result.get_counts()

        # Como as strings vêm em ordem c[2] c[1] c[0], o bit da tomografia é o primeiro caractere (bits[0])
        p0 = sum(cnt for bits, cnt in counts.items() if bits[0] == '0')
        p1 = sum(cnt for bits, cnt in counts.items() if bits[0] == '1')
        total = p0 + p1 or 1

        expectation_values[base] = (p0 - p1) / total
        all_counts[base] = counts

    # Reconstrução da matriz densidade
    rho = 0.5 * (
        pauli_list[0]
        + expectation_values['X'] * pauli_list[1]
        + expectation_values['Y'] * pauli_list[2]
        + expectation_values['Z'] * pauli_list[3]
    )

    return rho, expectation_values, circuit

#### Rho, valores esperados e circuitos

In [4]:
rho_0 ,expectation_values_0, circuit_0= function_tel_tomography(n=0,shots=10000)
print(rho_0)

rho_1 ,expectation_values_1, circuit_1= function_tel_tomography(n=1,shots=10000)
print(rho_1)

rho_2 ,expectation_values_2, circuit_2= function_tel_tomography(n=2,shots=10000)
print(rho_2)

rho_3 ,expectation_values_3, circuit_3= function_tel_tomography(n=3,shots=10000)
print(rho_3)

rho_4 ,expectation_values_4, circuit_4= function_tel_tomography(n=4,shots=10000)
print(rho_4)

rho_5 ,expectation_values_5, circuit_5= function_tel_tomography(n=5,shots=10000)
print(rho_5)

rho_6 ,expectation_values_6, circuit_6= function_tel_tomography(n=6,shots=10000)
print(rho_6)

rho_7 ,expectation_values_7, circuit_7= function_tel_tomography(n=7,shots=10000)
print(rho_7)

rho_8 ,expectation_values_8, circuit_8= function_tel_tomography(n=8,shots=10000)
print(rho_8)

rho_9 ,expectation_values_9, circuit_9= function_tel_tomography(n=9,shots=10000)
print(rho_9)

rho_10 ,expectation_values_10, circuit_10= function_tel_tomography(n=9,shots=10000)
print(rho_10)

[[ 1.    +0.j     -0.0081-0.0052j]
 [-0.0081+0.0052j  0.    +0.j    ]]
[[1.    +0.j     0.0018-0.0111j]
 [0.0018+0.0111j 0.    +0.j    ]]
[[ 1.    +0.j     -0.0011+0.0033j]
 [-0.0011-0.0033j  0.    +0.j    ]]
[[ 1.    +0.j     -0.0015-0.0066j]
 [-0.0015+0.0066j  0.    +0.j    ]]
[[1.e+00+0.j     9.e-04-0.0074j]
 [9.e-04+0.0074j 0.e+00+0.j    ]]
[[1.    +0.j     0.0074-0.0043j]
 [0.0074+0.0043j 0.    +0.j    ]]
[[1.    +0.j     0.0049-0.0005j]
 [0.0049+0.0005j 0.    +0.j    ]]
[[ 1.    +0.j     -0.0012-0.0026j]
 [-0.0012+0.0026j  0.    +0.j    ]]
[[ 1.    +0.j     -0.0038-0.0023j]
 [-0.0038+0.0023j  0.    +0.j    ]]
[[1.    +0.j     0.0044+0.0026j]
 [0.0044-0.0026j 0.    +0.j    ]]
[[ 1.e+00+0.j     -8.e-04+0.0026j]
 [-8.e-04-0.0026j  0.e+00+0.j    ]]
